In [2]:
!pip install -q -U bitsandbytes>=0.41.0 accelerate>=0.25.0

In [1]:
from datasets import load_dataset
import json

TRAIN_PATH = "/content/train.json"
TEST_PATH  = "/content/test.json"
train_raw = load_dataset("json", data_files=TRAIN_PATH, split="train")
test_raw  = load_dataset("json", data_files=TEST_PATH, split="train")
print("Ham Train:", len(train_raw))
print("Ham Test :", len(test_raw))

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Ham Train: 4503
Ham Test : 501


In [2]:
def is_valid_example(ex):
    if not ex.get("metin") or not ex["metin"].strip():
        return False
    if not ex.get("soru") or not ex["soru"].strip():
        return False
    if not ex.get("secenekler") or not isinstance(ex["secenekler"], list):
        return False
    if len(ex["secenekler"]) != 5:
        return False
    dogru = (ex.get("dogru_cevap") or "").strip()
    if dogru not in ["A", "B", "C", "D", "E"]:
        return False
    return True

# Temizleme
train_clean = train_raw.filter(is_valid_example)
test_clean  = test_raw.filter(is_valid_example)

print("Temiz Train:", len(train_clean))
print("Temiz Test :", len(test_clean))
print("Train silinen:", len(train_raw)-len(train_clean))
print("Test silinen :", len(test_raw)-len(test_clean))


Filter:   0%|          | 0/4503 [00:00<?, ? examples/s]

Filter:   0%|          | 0/501 [00:00<?, ? examples/s]

Temiz Train: 4503
Temiz Test : 500
Train silinen: 0
Test silinen : 1


In [3]:
# Normalization & Prompt üretimi
import re
def normalize_options(options):
    normalized = []
    for idx, opt in enumerate(options):
        letter = chr(ord("A") + idx)
        cleaned = re.sub(r"^[A-E][\)\:\.\-\s]*", "", opt).strip()
        normalized.append(f"{letter}: {cleaned}")
    return normalized


In [4]:
def make_prompt(ex):
    metin = ex["metin"].strip()
    soru = ex["soru"].strip()
    options_norm = normalize_options(ex["secenekler"])
    options_text = "\n".join(options_norm)
    dogru = ex["dogru_cevap"].strip()[0]
    aciklama = (ex.get("aciklama") or "").strip()

    prompt = f"""<|im_start|>system
Türkçe okuduğunu anlama uzmanısın. Metni dikkatlice oku, soruyu analiz et ve mantıklı gerekçeyle cevapla.<|im_end|>
<|im_start|>user
{metin}

Soru: {soru}
{options_text}<|im_end|>
<|im_start|>assistant
{aciklama} Dolayısıyla doğru cevap **{dogru}** şıkkıdır.<|im_end|>"""

    return {"text": prompt}

train_text = train_clean.map(make_prompt, remove_columns=train_clean.column_names)
test_text  = test_clean.map(make_prompt, remove_columns=test_clean.column_names)

print("\nÖrnek Train prompt:")
print(train_text[0]["text"])


Map:   0%|          | 0/4503 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]


Örnek Train prompt:
<|im_start|>system
Türkçe okuduğunu anlama uzmanısın. Metni dikkatlice oku, soruyu analiz et ve mantıklı gerekçeyle cevapla.<|im_end|>
<|im_start|>user
Koleksiyon ve koleksiyonculuk, eski çağlardan beri, tutku, sahiplenme ve koruma gibi çeşitli faktörlerle toplumsal hayatın içinde yer almış; dönemin kültürel yapısı ve inançlarına göre şekillenmiştir. Antik Dönem'de tannlara sunulan adaklarla başlayan biriktirme etkinliği inanç odaklı yapılmıştı. Roma İmparatorluğu Dönemi'nde ise Yunan heykel sanatıyla şekillenen ve sosyal geleneklerle bağdaşık bir anlam taşımıştır. Koleksiyonculuk, sıradan bir biriktirme ya da toplama eylemi değildir. Koleksiyoncu, bir tutku ya da hedefin peşinde bilinçli seçimler yapma çabasındadır. Koleksiyonculuk derin bilgi, deneyim ve düzenli bir çalışmayı gerektirir. Her aşamada, iyi bir koleksiyon oluşturamamak duyarlılığıyla yaşanan kaygılar vardır.

Soru: Bu parçada 'koleksiyonculuk'la ilgili aşağıdakilerden hangisine değinilmemiştir?
A: İ

In [5]:
# Tokenization
from transformers import AutoTokenizer

MODEL_NAME = "Qwen/Qwen3-14B"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

MAX_LEN = 512
def tokenize_fn(batch):
    enc = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)
    enc["labels"] = enc["input_ids"].copy()
    return enc

train_tok = train_text.map(tokenize_fn, batched=True, remove_columns=["text"])
val_tok   = test_text.map(tokenize_fn, batched=True, remove_columns=["text"])

print("Train tokens:", len(train_tok))
print("Val(Test) tokens:", len(val_tok))

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Map:   0%|          | 0/4503 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Train tokens: 4503
Val(Test) tokens: 500


In [6]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType, prepare_model_for_kbit_training


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    llm_int8_threshold=6.0,
    llm_int8_has_fp16_weight=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)


# İlk olarak kbit training için hazırla
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

# Gradient checkpointing
model.gradient_checkpointing_enable()

# Input gradients
model.enable_input_require_grads()

# Cache kapatma (training için gerekli)
model.config.use_cache = False


lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,

    r=64,
    lora_alpha=128,
    lora_dropout=0.1,

    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],

    bias="none",
    inference_mode=False,
    modules_to_save=None,

    init_lora_weights=True,
)

model = get_peft_model(model, lora_cfg)


trainable, total = model.get_nb_trainable_parameters()
print(f"{'='*60}")
print(f"🎯 Model: Qwen3-14B + LoRA")
print(f"{'='*60}")
print(f"📊 Trainable params: {trainable:,} ({trainable/total*100:.2f}%)")
print(f"📊 Total params:     {total:,}")
print(f"💾 Memory reduction: ~{(1 - trainable/total)*100:.1f}%")
print(f"{'='*60}")

model.print_trainable_parameters()

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/3.84G [00:00<?, ?B/s]

...

model-00002-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

🎯 Model: Qwen3-14B + LoRA
📊 Trainable params: 256,901,120 (1.71%)
📊 Total params:     15,025,208,320
💾 Memory reduction: ~98.3%
trainable params: 256,901,120 || all params: 15,025,208,320 || trainable%: 1.7098


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers.trainer_callback import EarlyStoppingCallback
import json
import time
import math


data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    # Output & Logging
    output_dir="./qwen3_parcada_lora",
    logging_dir="./qwen3_parcada_lora/logs",
    run_name="qwen3-turkish-qa-lora-a100",

    # Training Duration
    num_train_epochs=2,
    max_steps=-1,

    # 🚀 Batch
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,

    # Learning Rate Schedule
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    warmup_steps=0,

    # Regularization
    weight_decay=0.01,
    max_grad_norm=1.0,

    # Precision & Memory
    fp16=False,
    bf16=True,
    bf16_full_eval=True,
    tf32=True,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={"use_reentrant": False},

    # Optimization
    optim="adamw_torch_fused",
    adam_beta1=0.9,
    adam_beta2=0.999,
    adam_epsilon=1e-8,

    # Evaluation & Saving
    eval_strategy="epoch",
    eval_steps=None,
    save_strategy="epoch",
    save_steps=None,
    save_total_limit=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    # Logging
    logging_steps=10,
    logging_first_step=True,
    logging_nan_inf_filter=True,

    # Data Loading
    remove_unused_columns=False,
    dataloader_num_workers=4,
    dataloader_pin_memory=True,
    dataloader_prefetch_factor=2,
    dataloader_persistent_workers=True,

    # Misc
    report_to="tensorboard",
    seed=42,
    data_seed=42,
    disable_tqdm=False,

    # A100 / DDP
    ddp_find_unused_parameters=False,

    # Performance
    group_by_length=True,
    length_column_name="length",
    prediction_loss_only=False,
)

# ============================================
# 3. EARLY STOPPING (2 epoch'a uygun)
# ============================================
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=2,             # 2 epoch varken 3 mantıksız kalır
    early_stopping_threshold=0.0005,
)

# ============================================
# 4. TRAINER
# ============================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping],
)

print("🚀 Training başlıyor (80GB A100 Optimized: 2 epoch, BS=16)...")
print(f"📊 Train samples: {len(train_tok)}")
print(f"📊 Eval samples: {len(val_tok)}")
print(f"📊 Per device batch size: {training_args.per_device_train_batch_size}")
print(f"📊 Gradient accumulation: {training_args.gradient_accumulation_steps}")
effective_bs = training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps
print(f"📊 Effective batch size: {effective_bs}")

steps_per_epoch_est = math.ceil(len(train_tok) / effective_bs)
total_steps_est = steps_per_epoch_est * int(training_args.num_train_epochs)
print(f"📊 Steps/epoch (est): ~{steps_per_epoch_est}")
print(f"📊 Total steps (est): ~{total_steps_est}")

print(f"🚀 Optimizer: {training_args.optim}")
print("=" * 60)

start_time = time.time()
trainer.train()
training_time = time.time() - start_time

print("\n💾 Model kaydediliyor...")
save_dir = "./qwen3_parcada_lora_final"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

# Training history kaydet
with open(f"{save_dir}/training_history.json", "w", encoding="utf-8") as f:
    json.dump(trainer.state.log_history, f, indent=2, ensure_ascii=False)

# Metrics kaydet
metrics = trainer.evaluate()
metrics["training_time_seconds"] = training_time
metrics["training_time_hours"] = training_time / 3600

with open(f"{save_dir}/final_metrics.json", "w", encoding="utf-8") as f:
    json.dump(metrics, f, indent=2, ensure_ascii=False)

print("✅ Training tamamlandı!")
print(f"📊 Final Eval Loss: {metrics['eval_loss']:.4f}")
print(f"⏱️  Training time: {training_time/3600:.2f} hours")
print(f"🚀 Samples/second: {len(train_tok) * training_args.num_train_epochs / training_time:.2f}")


/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)
/tmp/ipython-input-3856793211.py:100: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with

🚀 Training başlıyor (80GB A100 Optimized: 2 epoch, BS=16)...
📊 Train samples: 4503
📊 Eval samples: 500
📊 Per device batch size: 16
📊 Gradient accumulation: 2
📊 Effective batch size: 32
📊 Steps/epoch (est): ~141
📊 Total steps (est): ~282
🚀 Optimizer: OptimizerNames.ADAMW_TORCH_FUSED


Epoch,Training Loss,Validation Loss
1,1.308300,1.310905
2,1.139700,1.298989



💾 Model kaydediliyor...


✅ Training tamamlandı!
📊 Final Eval Loss: 1.2991
⏱️  Training time: 0.92 hours
🚀 Samples/second: 2.71


In [ ]:
import re
import torch
from collections import defaultdict, Counter

LABELS = ["A", "B", "C", "D", "E"]

def build_inference_prompt(metin, soru, secenekler):
    options_norm = normalize_options(secenekler)
    options_text = "\n".join(options_norm)
    prompt = f"""Metin: {metin.strip()}

Soru: {soru.strip()}

Seçenekler:
{options_text}

Cevap:"""
    return prompt


def predict_letter_and_explanation(metin, soru, secenekler):
    prompt = build_inference_prompt(metin, soru, secenekler)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Bazı tokenizer'lar token_type_ids veriyor, causal LM'de gerek yok:
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    inputs = inputs.to(model.device)

    with torch.no_grad():
        out = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=4,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=None,     # istersen tokenizer.eos_token_id yapabilirsin
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # Cevap kısmını izole et
    if "Cevap:" in text:
        text = text.split("Cevap:", 1)[-1].strip()

    # Harfi yakala (A–E)
    m = re.search(r"\b([A-E])\b", text)
    letter = m.group(1) if m else None

    explanation = text  # açıklamanın tamamını açıklama olarak al

    return letter, explanation


def evaluate_model_on_dataset(test_dataset, limit=None, show_progress=True, top_k_errors=10):
    """
    test_dataset: test_clean (id, metin, soru, secenekler, dogru_cevap, aciklama içeren ham JSON kayıtları)
    limit: sadece ilk N soru için çalıştırmak istersen (ör: 50). Tamamı için None.
    """

    total = 0
    correct = 0

    confusion = {t: {p: 0 for p in LABELS + ["None"]} for t in LABELS}
    per_label_counts = Counter()
    mistake_list = []

    if show_progress:
        print("\n📌 Model değerlendirmesi başlıyor...\n")

    for idx, ex in enumerate(test_dataset):

        if limit and idx >= limit:
            break

        metin = ex["metin"]
        soru = ex["soru"]
        secenekler = ex["secenekler"]
        true_label = ex["dogru_cevap"].strip()

        pred_label, raw_output = predict_letter_and_explanation(
            metin,
            soru,
            secenekler
        )

        total += 1
        per_label_counts[true_label] += 1

        # None çıktısı olursa "None" sütununa yaz
        pred_effective = pred_label if pred_label in LABELS else "None"

        # confusion matrix'e işle
        if true_label in LABELS:
            confusion[true_label][pred_effective] += 1

        success = (pred_label == true_label)

        if success:
            correct += 1
        else:
            mistake_list.append({
                "index": idx,
                "id": ex.get("id", f"idx_{idx}"),
                "metin": metin,
                "soru": soru,
                "secenekler": secenekler,
                "true": true_label,
                "pred": pred_label,
                "output": raw_output
            })

        if show_progress:
            print(f"--- {idx+1}. Soru ---------------------------------")
            print(f"ID: {ex.get('id', f'idx_{idx}')}")
            print(f"Gerçek: {true_label} | Tahmin: {pred_label}")
            print("Durum:", "✔ DOĞRU" if success else "❌ YANLIŞ")
            print("Model Çıktısı:")
            print(raw_output)
            print("--------------------------------------------------\n")

    accuracy = correct / total if total > 0 else 0.0

    # === Rapor üretimi ===
    print("\n" + "="*70)
    print("🧾 MODEL EVALUATION REPORT")
    print("="*70)

    # 1) Genel istatistikler
    print("\n[1] GENEL İSTATİSTİKLER")
    print("-" * 70)
    print(f"Toplam test soru sayısı : {total}")
    print(f"Doğru cevap sayısı     : {correct}")
    print(f"Yanlış cevap sayısı    : {total - correct}")
    print(f"Accuracy               : {accuracy:.2%}")

    print("\nSınıfların (A/B/C/D/E) gerçek dağılımı:")
    for label in LABELS:
        print(f"  {label}: {per_label_counts[label]}")

    # 2) Confusion Matrix
    print("\n[2] CONFUSION MATRIX (Gerçek satır, Tahmin sütun)")
    print("-" * 70)
    header = ["Gerçek\\Tahmin"] + LABELS + ["None"]
    print(" | ".join(f"{h:>7}" for h in header))
    print("-" * (10 * (len(header)+1)))

    for t in LABELS:
        row = [f"{t:>7}"]
        for p in LABELS + ["None"]:
            row.append(f"{confusion[t][p]:>7}")
        print(" | ".join(row))

    # 3) Per-class Precision / Recall / F1
    print("\n[3] SINIF BAZLI METRİKLER (A/B/C/D/E)")
    print("-" * 70)
    print(f"{'Label':>5} | {'Support':>7} | {'Precision':>10} | {'Recall':>8} | {'F1':>6}")
    print("-" * 70)

    for label in LABELS:
        tp = confusion[label][label]
        fp = sum(confusion[other][label] for other in LABELS if other != label)
        fn = sum(confusion[label][other] for other in LABELS if other != label)

        support = per_label_counts[label]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

        print(f"{label:>5} | {support:>7} | {precision:>10.2%} | {recall:>8.2%} | {f1:>6.2%}")

    # 4) Yanlış sorular (özet)
    print("\n[4] YANLIŞ SORULAR ÖZETİ")
    print("-" * 70)
    print(f"Toplam yanlış soru: {len(mistake_list)}")
    print(f"İlk {min(top_k_errors, len(mistake_list))} tanesi gösteriliyor:\n")

    for m in mistake_list[:top_k_errors]:
        print(f"ID: {m['id']} (index: {m['index']})")
        print(f"Gerçek: {m['true']} | Tahmin: {m['pred']}")
        print("Soru:", m["soru"])
        print("Metin:", m["metin"][:200].replace("\n", " ") + ("..." if len(m["metin"]) > 200 else ""))
        print("Model Çıktısı:")
        print(m["output"])
        print("-" * 70)

    print("\n[5] ÖNERİLEN YORUM")
    print("-" * 70)
    print("• Accuracy'nin %80+ olması, modelin parçada anlam görevinde oldukça iyi performans gösterdiğini gösterir.")
    print("• Confusion matrix'te yoğunluk diyagonal üzerinde ise, model çoğunlukla doğru şıkkı seçiyor demektir.")
    print("• Belirli bir şıkta (örneğin C veya D) recall düşükse, o tür sorular için daha fazla örnekle ek eğitim yapılabilir.")
    print("• Model açıklamalarını da inceleyerek, mantık hatası mı yoksa metni yanlış anlama mı olduğuna karar verebilirsin.")
    print("="*70)

    return {
        "accuracy": accuracy,
        "mistakes": mistake_list,
        "confusion": confusion,
        "per_label_counts": per_label_counts
    }


# 🔚 Artık BURADA direkt test_clean'i kullanıyoruz (val_raw_ds yok!)
report = evaluate_model_on_dataset(test_clean, limit=None, show_progress=True, top_k_errors=10)



📌 Model değerlendirmesi başlıyor...

--- 1. Soru ---------------------------------
ID: soru_00910
Gerçek: C | Tahmin: C
Durum: ✔ DOĞRU
Model Çıktısı:
C şıkk
--------------------------------------------------

--- 2. Soru ---------------------------------
ID: soru_03683
Gerçek: A | Tahmin: A
Durum: ✔ DOĞRU
Model Çıktısı:
A şıkk
--------------------------------------------------

--- 3. Soru ---------------------------------
ID: soru_04493
Gerçek: B | Tahmin: B
Durum: ✔ DOĞRU
Model Çıktısı:
B şıkk
--------------------------------------------------

--- 4. Soru ---------------------------------
ID: soru_01225
Gerçek: B | Tahmin: B
Durum: ✔ DOĞRU
Model Çıktısı:
B şıkk
--------------------------------------------------

--- 5. Soru ---------------------------------
ID: soru_01272
Gerçek: C | Tahmin: C
Durum: ✔ DOĞRU
Model Çıktısı:
C
A)
--------------------------------------------------

--- 6. Soru ---------------------------------
ID: soru_01590
Gerçek: C | Tahmin: C
Durum: ✔ DOĞRU
Model 

In [ ]:
import re
import torch

def predict_letter_and_explanation(metin, soru, secenekler):
    prompt = build_inference_prompt(metin, soru, secenekler)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Qwen için token_type_ids silinmeli
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    inputs = inputs.to(model.device)

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=256,
            temperature=0.2,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # "Cevap:" sonrası kısmı izole et
    if "Cevap:" in text:
        text = text.split("Cevap:", 1)[-1].strip()

    # A–E harfi yakala
    m = re.search(r"\b([A-E])\b", text)
    letter = m.group(1) if m else None

    return letter, text


In [ ]:
test_ex = {
    "metin": """Erkek ve işçi yaban arıları kış mevsiminden önce ölür, kraliçe yaban arıları ise kış uykusuna yatar. Bu durum onların aylarca soğuğa dayanmalarına ve baharda uyanarak yeni bir koloni kurmalarına olanak tanır. Kış boyunca hayatta kalan kraliçe arıların sayısı, ekosistemlerin sürdürülebilirliği ile doğrudan ilişkili olduğundan büyük önem taşır ve bu nedenle birçok çalışmanın odak noktasını oluşturur. Yakın zamanda yapılan bir deney esnasında da yanlışlıkla suya düşen kraliçe arıların önce öldüğü düşünülürken su boşaltıldığında uyandıkları ve herhangi bir zarar görmedikleri gözlemlenmiştir. Bu durum, ekosistem için önemli olan arıların olası bir su baskınına fiziksel olarak adapte olabilmelerinin mümkün olduğunu göstermiştir.""",

    "soru": "Bu parçadan kraliçe yaban arılarıyla ilgili aşağıdakilerin hangisine ulaşılabilir?",

    "secenekler": [
        "A) Geleceklerinin su altında yaşama uyum sağlamalarına bağlı olduğuna",
        "B) Koloni içindeki güç ve otoritelerinin araştırmalara konu edildiğine",
        "C) Ilıman iklime sahip bölgelerde daha üretken bir yaşam sürdüğüne",
        "D) Kış uykusuna yatmalarının ekosistemin devamlılığında kritik rol oynadığına",
        "E) Ekosistemdeki bitki çeşitlerinin artmasında önemli bir rol üstlendiğine",
    ],

    "dogru_cevap": "D",
}

pred_letter, explanation = predict_letter_and_explanation(
    test_ex["metin"],
    test_ex["soru"],
    test_ex["secenekler"],
)

print("Model cevabı:\n", explanation)
print("\nTahmin edilen harf:", pred_letter, "| Gerçek:", test_ex["dogru_cevap"])


Model cevabı:
 D şıkkı doğrudur; çünkü metinde kraliçe arıların kış uykusuna yatmalarının ekosistemlerin sürdürülebilirliğiyle doğrudan ilişkili olduğu ve bu nedenle birçok çalışmanın odak noktası olduğu belirtilmektedir. A şıkkı yanlıştır; çünkü metinde geleceklerinin su altında yaşama uyumuna bağlı olduğu ifade edilmemektedir. B şıkkı yanlıştır; çünkü metinde koloni içindeki güç ve otoriteyle ilgili bir araştırma yapıldığına dair bir bilgi verilmemektedir. C şıkkı yanlıştır; çünkü metinde ilıman iklime sahip bölgelerde üretkenlikten bahsedilmemektedir. E şıkkı yanlıştır; çünkü metinde ekosistemdeki bitki çeşitlerinin artmasında kraliçe arıların rolüne dair bir bilgi verilmemektedir. Dolayısıyla doğru cevap **D** şıkkıdır.

Tahmin edilen harf: D | Gerçek: D


In [9]:
import re
import torch
from collections import Counter
import time
from tqdm import tqdm

# ============================================
# HELPER FUNCTIONS
# ============================================

LABELS = ["A", "B", "C", "D", "E"]

def normalize_options(options):
    """Seçenekleri A:, B:, C:, D:, E: formatına normalize et"""
    normalized = []
    for idx, opt in enumerate(options):
        letter = chr(ord("A") + idx)
        cleaned = re.sub(r"^[A-E][\)\:\.\-\s]*", "", opt).strip()
        normalized.append(f"{letter}: {cleaned}")
    return normalized


def build_inference_prompt(metin, soru, secenekler):
    """Inference için prompt oluştur"""
    options_norm = normalize_options(secenekler)
    options_text = "\n".join(options_norm)

    prompt = f"""<|im_start|>system
Türkçe okuduğunu anlama uzmanısın. Metni dikkatlice oku, soruyu analiz et ve mantıklı gerekçeyle cevapla.<|im_end|>
<|im_start|>user
{metin.strip()}

Soru: {soru.strip()}

Seçenekler:
{options_text}<|im_end|>
<|im_start|>assistant
"""
    return prompt


def predict_letter_and_explanation(model, tokenizer, metin, soru, secenekler):
    """Model ile tahmin yap ve harfi + açıklamayı döndür"""
    prompt = build_inference_prompt(metin, soru, secenekler)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Token_type_ids varsa sil (causal LM için gerekli değil)
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    inputs = inputs.to(model.device)

    with torch.no_grad():
        out = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=8,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    text = tokenizer.decode(out[0], skip_special_tokens=True)

    # Assistant çıktısını izole et
    if "<|im_start|>assistant" in text:
        text = text.split("<|im_start|>assistant", 1)[-1].strip()
    if "<|im_end|>" in text:
        text = text.split("<|im_end|>", 1)[0].strip()

    # Harfi yakala (A-E arasında)
    m = re.search(r"\*\*([A-E])\*\*|\b([A-E])\b", text)
    if m:
        letter = m.group(1) if m.group(1) else m.group(2)
    else:
        letter = None

    return letter, text


# ============================================
# EVALUATION FUNCTION WITH DETAILED LOGGING
# ============================================

def evaluate_pretrained_model(model, tokenizer, test_dataset, limit=None, show_progress=True,
                               top_k_errors=10, log_every=10):
    """
    Pre-trained (fine-tune öncesi) modeli test verisinde değerlendir

    Args:
        model: Yüklenmiş model
        tokenizer: Tokenizer
        test_dataset: Test veri seti (test_clean)
        limit: Kaç örnek değerlendirilecek (None = tümü)
        show_progress: Her örneği ekrana yazdır
        top_k_errors: Kaç hatalı örnek detaylı gösterilecek
        log_every: Kaç örnek başına ara log yazdırılacak

    Returns:
        dict: accuracy, mistakes, confusion matrix ve diğer metrikler
    """

    print("\n" + "="*80)
    print("🔍 PRE-TRAINING MODEL EVALUATION (DETAILED LOGGING)")
    print("="*80)
    print(f"🤖 Model: {model.config._name_or_path if hasattr(model.config, '_name_or_path') else 'Unknown'}")
    print(f"📊 Test samples: {len(test_dataset) if limit is None else min(limit, len(test_dataset))}")
    print(f"⚙️  Device: {next(model.parameters()).device}")
    print(f"📝 Logging every: {log_every} samples")
    print("="*80 + "\n")

    total = 0
    correct = 0

    # Confusion matrix için
    confusion = {t: {p: 0 for p in LABELS + ["None"]} for t in LABELS}
    per_label_counts = Counter()
    mistake_list = []

    # Timing
    start_time = time.time()
    inference_times = []

    # Model'i eval moduna al
    model.eval()

    # Progress bar
    total_samples = len(test_dataset) if limit is None else min(limit, len(test_dataset))
    pbar = tqdm(enumerate(test_dataset), total=total_samples, desc="🔄 Evaluating", ncols=100)

    for idx, ex in pbar:
        if limit and idx >= limit:
            break

        metin = ex["metin"]
        soru = ex["soru"]
        secenekler = ex["secenekler"]
        true_label = ex["dogru_cevap"].strip()

        # Inference timing
        inf_start = time.time()
        pred_label, raw_output = predict_letter_and_explanation(
            model, tokenizer, metin, soru, secenekler
        )
        inf_time = time.time() - inf_start
        inference_times.append(inf_time)

        total += 1
        per_label_counts[true_label] += 1

        # None çıktısı kontrolü
        pred_effective = pred_label if pred_label in LABELS else "None"

        # Confusion matrix güncelle
        if true_label in LABELS:
            confusion[true_label][pred_effective] += 1

        success = (pred_label == true_label)

        if success:
            correct += 1
        else:
            mistake_list.append({
                "index": idx,
                "id": ex.get("id", f"idx_{idx}"),
                "metin": metin,
                "soru": soru,
                "secenekler": secenekler,
                "true": true_label,
                "pred": pred_label,
                "output": raw_output,
                "inference_time": inf_time
            })

        # Update progress bar
        current_acc = correct / total if total > 0 else 0.0
        pbar.set_postfix({
            'Acc': f'{current_acc:.2%}',
            'Correct': correct,
            'Wrong': total - correct
        })

        # Periyodik detaylı log
        if (idx + 1) % log_every == 0:
            elapsed = time.time() - start_time
            avg_time = sum(inference_times) / len(inference_times)
            current_acc = correct / total

            print(f"\n{'='*80}")
            print(f"📊 CHECKPOINT @ Sample {idx + 1}/{total_samples}")
            print(f"{'='*80}")
            print(f"✅ Doğru: {correct}/{total} ({current_acc:.2%})")
            print(f"❌ Yanlış: {total - correct}/{total} ({(total-correct)/total:.2%})")
            print(f"⚠️  None tahminleri: {sum(confusion[t]['None'] for t in LABELS)}")
            print(f"⏱️  Ortalama inference: {avg_time:.3f}s/sample")
            print(f"⏱️  Toplam süre: {elapsed:.1f}s ({elapsed/60:.1f} dakika)")
            print(f"🚀 Kalan süre tahmini: {(total_samples - (idx+1)) * avg_time / 60:.1f} dakika")
            print(f"{'='*80}\n")

        # Detaylı soru logu (opsiyonel)
        if show_progress:
            print(f"\n{'─'*80}")
            print(f"📌 Soru #{idx+1}")
            print(f"{'─'*80}")
            print(f"🆔 ID: {ex.get('id', f'idx_{idx}')}")
            print(f"📖 Metin: {metin[:150]}..." if len(metin) > 150 else f"📖 Metin: {metin}")
            print(f"❓ Soru: {soru}")
            print(f"📝 Seçenekler:")
            for i, sec in enumerate(secenekler):
                print(f"   {chr(65+i)}: {sec}")
            print(f"\n🎯 Gerçek Cevap: {true_label}")
            print(f"🤖 Model Tahmini: {pred_label if pred_label else 'None'}")
            print(f"{'✅ DOĞRU' if success else '❌ YANLIŞ'}")
            print(f"\n💬 Model Çıktısı:")
            print(f"   {raw_output[:200]}{'...' if len(raw_output) > 200 else ''}")
            print(f"\n⏱️  Inference süresi: {inf_time:.3f}s")
            print(f"{'─'*80}\n")

    pbar.close()

    total_time = time.time() - start_time
    accuracy = correct / total if total > 0 else 0.0
    avg_inference_time = sum(inference_times) / len(inference_times) if inference_times else 0

    # ============================================
    # DETAYLI RAPOR OLUŞTURMA
    # ============================================

    print("\n" + "="*80)
    print("📊 FINAL PRE-TRAINING EVALUATION REPORT")
    print("="*80)

    # 1) Genel istatistikler
    print("\n[1] GENEL İSTATİSTİKLER")
    print("-" * 80)
    print(f"Toplam test soru sayısı    : {total}")
    print(f"Doğru cevap sayısı         : {correct}")
    print(f"Yanlış cevap sayısı        : {total - correct}")
    print(f"None tahmin sayısı         : {sum(confusion[t]['None'] for t in LABELS)}")
    print(f"Accuracy                   : {accuracy:.2%}")
    print(f"\n⏱️  ZAMANLAMA BİLGİLERİ:")
    print(f"Toplam değerlendirme süresi: {total_time:.1f}s ({total_time/60:.2f} dakika)")
    print(f"Ortalama inference süresi  : {avg_inference_time:.3f}s/sample")
    print(f"Throughput                 : {total/total_time:.2f} samples/s")

    print("\n📊 Sınıfların (A/B/C/D/E) gerçek dağılımı:")
    for label in LABELS:
        count = per_label_counts[label]
        percentage = (count / total * 100) if total > 0 else 0
        print(f"  {label}: {count:4d} ({percentage:5.1f}%)")

    # 2) Confusion Matrix
    print("\n[2] CONFUSION MATRIX (Gerçek satır, Tahmin sütun)")
    print("-" * 80)
    header = ["True\\Pred"] + LABELS + ["None"]
    print(" | ".join(f"{h:>8}" for h in header))
    print("-" * 80)

    for t in LABELS:
        row = [f"{t:>8}"]
        for p in LABELS + ["None"]:
            count = confusion[t][p]
            row.append(f"{count:>8}")
        print(" | ".join(row))

    # 3) Per-class Precision / Recall / F1
    print("\n[3] SINIF BAZLI METRİKLER (A/B/C/D/E)")
    print("-" * 80)
    print(f"{'Label':>5} | {'Support':>8} | {'Precision':>10} | {'Recall':>8} | {'F1':>8}")
    print("-" * 80)

    macro_precision, macro_recall, macro_f1 = 0, 0, 0

    for label in LABELS:
        tp = confusion[label][label]
        fp = sum(confusion[other][label] for other in LABELS if other != label)
        fn = sum(confusion[label][other] for other in LABELS + ["None"] if other != label)

        support = per_label_counts[label]

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        recall    = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) > 0 else 0.0

        macro_precision += precision
        macro_recall += recall
        macro_f1 += f1

        print(f"{label:>5} | {support:>8} | {precision:>10.2%} | {recall:>8.2%} | {f1:>8.2%}")

    # Macro averages
    macro_precision /= len(LABELS)
    macro_recall /= len(LABELS)
    macro_f1 /= len(LABELS)

    print("-" * 80)
    print(f"{'MACRO':>5} | {'AVG':>8} | {macro_precision:>10.2%} | {macro_recall:>8.2%} | {macro_f1:>8.2%}")

    # 4) En çok karıştırılan şıklar
    print("\n[4] EN ÇOK KARIŞTIRILAN ŞIK ÇİFTLERİ")
    print("-" * 80)
    confusion_pairs = []
    for true_label in LABELS:
        for pred_label in LABELS + ["None"]:
            if true_label != pred_label and confusion[true_label][pred_label] > 0:
                confusion_pairs.append((
                    true_label,
                    pred_label,
                    confusion[true_label][pred_label]
                ))

    confusion_pairs.sort(key=lambda x: x[2], reverse=True)
    print("En çok karıştırılan 10 çift:")
    for i, (true_l, pred_l, count) in enumerate(confusion_pairs[:10], 1):
        print(f"{i:2d}. {true_l} → {pred_l}: {count:3d} kez")

    # 5) Yanlış sorular (özet)
    print("\n[5] YANLIŞ SORULAR ÖZETİ")
    print("-" * 80)
    print(f"Toplam yanlış soru: {len(mistake_list)}")
    print(f"İlk {min(top_k_errors, len(mistake_list))} tanesi gösteriliyor:\n")

    for i, m in enumerate(mistake_list[:top_k_errors], 1):
        print(f"{'─'*80}")
        print(f"❌ Hata #{i}")
        print(f"{'─'*80}")
        print(f"🆔 ID: {m['id']} (index: {m['index']})")
        print(f"🎯 Gerçek: {m['true']} | 🤖 Tahmin: {m['pred'] if m['pred'] else 'None'}")
        print(f"❓ Soru: {m['soru'][:100]}{'...' if len(m['soru']) > 100 else ''}")
        print(f"📖 Metin: {m['metin'][:200].replace(chr(10), ' ')}{'...' if len(m['metin']) > 200 else ''}")
        print(f"💬 Model Çıktısı: {m['output'][:200]}{'...' if len(m['output']) > 200 else ''}")
        print(f"⏱️  Inference: {m['inference_time']:.3f}s")

    # 6) Baseline değerlendirme ve öneriler
    print("\n[6] BASELINE (PRE-TRAINING) DEĞERLENDİRME")
    print("-" * 80)
    print(f"🎯 Baseline Accuracy: {accuracy:.2%}")

    if accuracy < 0.20:
        status = "🔴 ÇOK ZAYIF"
        comment = "Model neredeyse rastgele tahmin yapıyor. Fine-tuning kritik!"
    elif accuracy < 0.30:
        status = "🟠 ZAYIF"
        comment = "Model çok zayıf. Fine-tuning ile dramatik gelişme beklenir."
    elif accuracy < 0.50:
        status = "🟡 DÜŞÜK"
        comment = "Model düşük performans gösteriyor. Fine-tuning şart."
    elif accuracy < 0.70:
        status = "🟢 ORTA"
        comment = "Model orta seviye performans gösteriyor. Fine-tuning ile iyileşir."
    elif accuracy < 0.85:
        status = "🟢 İYİ"
        comment = "Model iyi performans gösteriyor. Fine-tuning ile optimize edilebilir."
    else:
        status = "🟢 ÇOK İYİ"
        comment = "Model çok iyi! Fine-tuning ile mükemmelleşebilir."

    print(f"Durum: {status}")
    print(f"Yorum: {comment}")

    none_ratio = sum(confusion[t]['None'] for t in LABELS) / total if total > 0 else 0
    print(f"\n⚠️  None oranı: {none_ratio:.2%}")
    if none_ratio > 0.20:
        print("   → None oranı yüksek! Model cevap formatını öğrenmemiş.")
    elif none_ratio > 0.05:
        print("   → Makul None oranı. Fine-tuning ile düşecek.")
    else:
        print("   → Düşük None oranı. Model formatı anlıyor.")

    print("\n💡 ÖNERİLER:")
    print("• Bu baseline sonuçlarını not alın ve fine-tuning sonrası karşılaştırın.")
    print("• Confusion matrix'te en çok karıştırılan şıklara dikkat edin.")
    print("• Fine-tuning sonrası en az %10-20 accuracy artışı beklenir.")
    print("• Hatalı örnekleri inceleyin - veri kalitesi problemi olabilir.")
    print("="*80 + "\n")

    return {
        "accuracy": accuracy,
        "total": total,
        "correct": correct,
        "wrong": total - correct,
        "none_predictions": sum(confusion[t]['None'] for t in LABELS),
        "mistakes": mistake_list,
        "confusion": confusion,
        "per_label_counts": per_label_counts,
        "macro_precision": macro_precision,
        "macro_recall": macro_recall,
        "macro_f1": macro_f1,
        "total_time": total_time,
        "avg_inference_time": avg_inference_time,
        "throughput": total / total_time
    }


# ============================================
# KULLANIM
# ============================================

print("🚀 Pre-training evaluation başlıyor...")
print("📝 Model ve tokenizer değişkenlerinin yüklü olduğundan emin olun.\n")

# Önce küçük bir test (ilk 20 örnek) - hızlı kontrol
print("\n" + "="*80)
print("🔬 PHASE 1: Quick Test (First 20 samples)")
print("="*80)
quick_test = evaluate_pretrained_model(
    model=model,
    tokenizer=tokenizer,
    test_dataset=test_clean,
    limit=20,
    show_progress=False,  # Detaylı log kapalı
    top_k_errors=3,
    log_every=5
)

print(f"\n✅ Quick test tamamlandı!")
print(f"📊 Quick Accuracy: {quick_test['accuracy']:.2%}")
print(f"⏱️  Average inference time: {quick_test['avg_inference_time']:.3f}s")

# Devam etmek istiyor musunuz?
print("\n" + "="*80)
response = input("🤔 Tam değerlendirmeye devam etmek istiyor musunuz? (y/n): ")
print("="*80 + "\n")

if response.lower() in ['y', 'yes', 'e', 'evet']:
    print("🚀 PHASE 2: Full Evaluation başlıyor...\n")

    # Tam değerlendirme
    baseline_results = evaluate_pretrained_model(
        model=model,
        tokenizer=tokenizer,
        test_dataset=test_clean,
        limit=None,  # Tüm veri
        show_progress=False,  # Detaylı log kapalı (performans için)
        top_k_errors=15,
        log_every=50  # Her 50 örnekte bir checkpoint
    )

    # Sonuçları kaydet
    import json
    print("\n💾 Sonuçlar kaydediliyor...")

    with open("baseline_evaluation_results.json", "w", encoding="utf-8") as f:
        # Mistakes listesini kısalt (dosya boyutu için)
        save_results = {
            "accuracy": baseline_results["accuracy"],
            "total": baseline_results["total"],
            "correct": baseline_results["correct"],
            "wrong": baseline_results["wrong"],
            "none_predictions": baseline_results["none_predictions"],
            "macro_precision": baseline_results["macro_precision"],
            "macro_recall": baseline_results["macro_recall"],
            "macro_f1": baseline_results["macro_f1"],
            "total_time": baseline_results["total_time"],
            "avg_inference_time": baseline_results["avg_inference_time"],
            "throughput": baseline_results["throughput"],
            "per_label_counts": dict(baseline_results["per_label_counts"]),
            "confusion_matrix": baseline_results["confusion"],
            "top_20_mistakes": baseline_results["mistakes"][:20]  # İlk 20 hata
        }
        json.dump(save_results, f, indent=2, ensure_ascii=False)

    print("✅ Baseline evaluation tamamlandı!")
    print(f"📊 Sonuçlar 'baseline_evaluation_results.json' dosyasına kaydedildi.")
    print(f"\n🎯 FINAL BASELINE ACCURACY: {baseline_results['accuracy']:.2%}")
    print(f"⏱️  Total time: {baseline_results['total_time']/60:.2f} minutes")
    print(f"🚀 Throughput: {baseline_results['throughput']:.2f} samples/second")
else:
    print("⏸️  Tam değerlendirme iptal edildi. Quick test sonuçlarını kullanabilirsiniz.")

🚀 Pre-training evaluation başlıyor...
📝 Model ve tokenizer değişkenlerinin yüklü olduğundan emin olun.


🔬 PHASE 1: Quick Test (First 20 samples)

🔍 PRE-TRAINING MODEL EVALUATION (DETAILED LOGGING)
🤖 Model: Qwen/Qwen3-14B
📊 Test samples: 20
⚙️  Device: cuda:0
📝 Logging every: 5 samples



🔄 Evaluating:  25%|████▎            | 5/20 [00:07<00:22,  1.51s/it, Acc=20.00%, Correct=1, Wrong=4]


📊 CHECKPOINT @ Sample 5/20
✅ Doğru: 1/5 (20.00%)
❌ Yanlış: 4/5 (80.00%)
⚠️  None tahminleri: 0
⏱️  Ortalama inference: 1.514s/sample
⏱️  Toplam süre: 7.6s (0.1 dakika)
🚀 Kalan süre tahmini: 0.4 dakika



🔄 Evaluating:  50%|████████        | 10/20 [00:15<00:15,  1.51s/it, Acc=10.00%, Correct=1, Wrong=9]


📊 CHECKPOINT @ Sample 10/20
✅ Doğru: 1/10 (10.00%)
❌ Yanlış: 9/10 (90.00%)
⚠️  None tahminleri: 0
⏱️  Ortalama inference: 1.507s/sample
⏱️  Toplam süre: 15.1s (0.3 dakika)
🚀 Kalan süre tahmini: 0.3 dakika



...


📊 CHECKPOINT @ Sample 400/500
✅ Doğru: 81/400 (20.25%)
❌ Yanlış: 319/400 (79.75%)
⚠️  None tahminleri: 0
⏱️  Ortalama inference: 1.489s/sample
⏱️  Toplam süre: 596.5s (9.9 dakika)
🚀 Kalan süre tahmini: 2.5 dakika



🔄 Evaluating:  90%|█████████▉ | 450/500 [11:10<01:13,  1.48s/it, Acc=20.44%, Correct=92, Wrong=358]


📊 CHECKPOINT @ Sample 450/500
✅ Doğru: 92/450 (20.44%)
❌ Yanlış: 358/450 (79.56%)
⚠️  None tahminleri: 0
⏱️  Ortalama inference: 1.489s/sample
⏱️  Toplam süre: 670.7s (11.2 dakika)
🚀 Kalan süre tahmini: 1.2 dakika



🔄 Evaluating: 100%|██████████| 500/500 [12:24<00:00,  1.49s/it, Acc=21.20%, Correct=106, Wrong=394]


📊 CHECKPOINT @ Sample 500/500
✅ Doğru: 106/500 (21.20%)
❌ Yanlış: 394/500 (78.80%)
⚠️  None tahminleri: 0
⏱️  Ortalama inference: 1.487s/sample
⏱️  Toplam süre: 744.6s (12.4 dakika)
🚀 Kalan süre tahmini: 0.0 dakika


📊 FINAL PRE-TRAINING EVALUATION REPORT

[1] GENEL İSTATİSTİKLER
--------------------------------------------------------------------------------
Toplam test soru sayısı    : 500
Doğru cevap sayısı         : 106
Yanlış cevap sayısı        : 394
None tahmin sayısı         : 0
Accuracy                   : 21.20%

⏱️  ZAMANLAMA BİLGİLERİ:
Toplam değerlendirme süresi: 744.6s (12.41 dakika)
Ortalama inference süresi  : 1.487s/sample
Throughput                 : 0.67 samples/s

📊 Sınıfların (A/B/C/D/E) gerçek dağılımı:
  A:  106 ( 21.2%)
  B:   69 ( 13.8%)
  C:  110 ( 22.0%)
  D:  105 ( 21.0%)
  E:  110 ( 22.0%)

[2] CONFUSION MATRIX (Gerçek satır, Tahmin sütun)
--------------------------------------------------------------------------------
True\Pred |        A |        B |    

In [11]:
import re
import torch

# ============================================
# HELPER FUNCTIONS
# ============================================

def normalize_options(options):
    """Seçenekleri A:, B:, C:, D:, E: formatına normalize et"""
    normalized = []
    for idx, opt in enumerate(options):
        letter = chr(ord("A") + idx)
        cleaned = re.sub(r"^[A-E][\)\:\.\-\s]*", "", opt).strip()
        normalized.append(f"{letter}: {cleaned}")
    return normalized


def build_inference_prompt(metin, soru, secenekler):
    """Inference için prompt oluştur"""
    options_norm = normalize_options(secenekler)
    options_text = "\n".join(options_norm)

    prompt = f"""<|im_start|>system
Türkçe okuduğunu anlama uzmanısın. Metni dikkatlice oku, soruyu analiz et ve mantıklı gerekçeyle cevapla.<|im_end|>
<|im_start|>user
{metin.strip()}

Soru: {soru.strip()}

Seçenekler:
{options_text}<|im_end|>
<|im_start|>assistant
"""
    return prompt


def predict_letter_and_explanation(model, tokenizer, metin, soru, secenekler, max_new_tokens=256):
    """
    Model ile tahmin yap ve harfi + açıklamayı döndür

    Args:
        model: Model
        tokenizer: Tokenizer
        metin: Paragraf metni
        soru: Soru
        secenekler: Şıklar listesi
        max_new_tokens: Maksimum üretilecek token sayısı

    Returns:
        tuple: (tahmin_harfi, model_çıktısı)
    """
    prompt = build_inference_prompt(metin, soru, secenekler)
    inputs = tokenizer(prompt, return_tensors="pt")

    # Token_type_ids varsa sil (causal LM için gerekli değil)
    if "token_type_ids" in inputs:
        del inputs["token_type_ids"]

    inputs = inputs.to(model.device)

    print("🔄 Model inference başlıyor...")
    print(f"📊 Input token sayısı: {inputs['input_ids'].shape[1]}")
    print(f"📝 Max new tokens: {max_new_tokens}")

    with torch.no_grad():
        out = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            temperature=0.3,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )

    full_text = tokenizer.decode(out[0], skip_special_tokens=True)

    print(f"✅ Inference tamamlandı!")
    print(f"📊 Output token sayısı: {out.shape[1]}")
    print(f"📊 Generated tokens: {out.shape[1] - inputs['input_ids'].shape[1]}")

    # Assistant çıktısını izole et
    assistant_output = full_text
    if "<|im_start|>assistant" in full_text:
        assistant_output = full_text.split("<|im_start|>assistant", 1)[-1].strip()
    if "<|im_end|>" in assistant_output:
        assistant_output = assistant_output.split("<|im_end|>", 1)[0].strip()

    # Harfi yakala (A-E arasında)
    # Önce **A** formatını ara, sonra tek başına harfi
    m = re.search(r"\*\*([A-E])\*\*", assistant_output)
    if not m:
        m = re.search(r"\b([A-E])\b", assistant_output)

    letter = m.group(1) if m else None

    return letter, assistant_output, full_text


# ============================================
# TEST EXAMPLE
# ============================================

test_ex = {
    "metin": """Erkek ve işçi yaban arıları kış mevsiminden önce ölür, kraliçe yaban arıları ise kış uykusuna yatar. Bu durum onların aylarca soğuğa dayanmalarına ve baharda uyanarak yeni bir koloni kurmalarına olanak tanır. Kış boyunca hayatta kalan kraliçe arıların sayısı, ekosistemlerin sürdürülebilirliği ile doğrudan ilişkili olduğundan büyük önem taşır ve bu nedenle birçok çalışmanın odak noktasını oluşturur. Yakın zamanda yapılan bir deney esnasında da yanlışlıkla suya düşen kraliçe arıların önce öldüğü düşünülürken su boşaltıldığında uyandıkları ve herhangi bir zarar görmedikleri gözlemlenmiştir. Bu durum, ekosistem için önemli olan arıların olası bir su baskınına fiziksel olarak adapte olabilmelerinin mümkün olduğunu göstermiştir.""",
    "soru": "Bu parçadan kraliçe yaban arılarıyla ilgili aşağıdakilerin hangisine ulaşılabilir?",
    "secenekler": [
        "A) Geleceklerinin su altında yaşama uyum sağlamalarına bağlı olduğuna",
        "B) Koloni içindeki güç ve otoritelerinin araştırmalara konu edildiğine",
        "C) Ilıman iklime sahip bölgelerde daha üretken bir yaşam sürdüğüne",
        "D) Kış uykusuna yatmalarının ekosistemin devamlılığında kritik rol oynadığına",
        "E) Ekosistemdeki bitki çeşitlerinin artmasında önemli bir rol üstlendiğine",
    ],
    "dogru_cevap": "D",
}


# ============================================
# DETAILED EVALUATION
# ============================================

print("\n" + "="*80)
print("🧪 SINGLE EXAMPLE DETAILED EVALUATION")
print("="*80)

print("\n📖 METIN:")
print("-" * 80)
print(test_ex["metin"])

print("\n❓ SORU:")
print("-" * 80)
print(test_ex["soru"])

print("\n📝 SEÇENEKLER:")
print("-" * 80)
for i, sec in enumerate(test_ex["secenekler"]):
    print(f"{chr(65+i)}) {sec}")

print("\n🎯 DOĞRU CEVAP:")
print("-" * 80)
print(f"✅ {test_ex['dogru_cevap']}")

print("\n" + "="*80)
print("🤖 MODEL EVALUATION (max_new_tokens=256)")
print("="*80)

# Model ile tahmin yap
pred_letter, assistant_output, full_output = predict_letter_and_explanation(
    model=model,
    tokenizer=tokenizer,
    metin=test_ex["metin"],
    soru=test_ex["soru"],
    secenekler=test_ex["secenekler"],
    max_new_tokens=512
)

print("\n💬 MODEL ÇIKTISI (Assistant Output):")
print("-" * 80)
print(assistant_output)

print("\n📊 SONUÇ ANALİZİ:")
print("-" * 80)
print(f"🤖 Model Tahmini: {pred_letter if pred_letter else '❌ None (Harf bulunamadı)'}")
print(f"🎯 Doğru Cevap  : {test_ex['dogru_cevap']}")

if pred_letter == test_ex['dogru_cevap']:
    print(f"\n✅ BAŞARILI! Model doğru cevabı buldu.")
    success_emoji = "🎉"
elif pred_letter is None:
    print(f"\n❌ BAŞARISIZ! Model geçerli bir harf üretemedi.")
    success_emoji = "⚠️"
else:
    print(f"\n❌ BAŞARISIZ! Model yanlış cevap verdi.")
    success_emoji = "❌"

print("\n" + "="*80)
print(f"{success_emoji} Değerlendirme Tamamlandı!")
print("="*80)


# ============================================
# OPTIONAL: Show full output with prompt
# ============================================

show_full = input("\n🤔 Tam çıktıyı (prompt dahil) görmek ister misiniz? (y/n): ")
if show_full.lower() in ['y', 'yes', 'e', 'evet']:
    print("\n📋 TAM ÇIKTI (Prompt + Model Response):")
    print("="*80)
    print(full_output)
    print("="*80)


# ============================================
# ANALYSIS
# ============================================

print("\n📊 DETAYLI ANALİZ:")
print("-" * 80)

# Harf pozisyonu analizi
if pred_letter:
    letter_pos = assistant_output.find(pred_letter)
    print(f"✓ Tahmin edilen harf '{pred_letter}' çıktının {letter_pos}. karakterinde bulundu")
else:
    print("✗ Çıktıda geçerli bir harf (A-E) bulunamadı")

# Kelime sayısı
word_count = len(assistant_output.split())
print(f"✓ Model çıktısı {word_count} kelime içeriyor")

# Token analizi
output_tokens = tokenizer.encode(assistant_output)
print(f"✓ Model çıktısı yaklaşık {len(output_tokens)} token")

# Şık analizi - hangi şıklardan bahsediyor?
print(f"\n🔍 Model çıktısında bahsedilen şıklar:")
for letter in ["A", "B", "C", "D", "E"]:
    if letter in assistant_output:
        count = assistant_output.count(letter)
        print(f"  {letter}: {count} kez geçiyor")

# Anahtar kelime analizi
keywords = ["kış uykusu", "ekosistem", "sürdürülebilir", "koloni", "su baskını", "adapte"]
print(f"\n🔑 Anahtar kelimeler:")
for kw in keywords:
    if kw.lower() in assistant_output.lower():
        print(f"  ✓ '{kw}' bahsedilmiş")
    else:
        print(f"  ✗ '{kw}' bahsedilmemiş")

print("\n" + "="*80)

# Doğru cevap gerekçesi
print("\n💡 DOĞRU CEVAP GEREKÇESİ:")
print("-" * 80)
print("D şıkkı doğru çünkü:")
print("1. Parçada 'Kış boyunca hayatta kalan kraliçe arıların sayısı,")
print("   ekosistemlerin sürdürülebilirliği ile doğrudan ilişkili' deniliyor.")
print("2. Bu, kış uykusunun ekosistemin devamlılığında kritik rol oynadığını gösterir.")
print("3. Kraliçe arılar kış uykusuna yatarak baharda yeni koloni kurarlar.")
print("="*80)


🧪 SINGLE EXAMPLE DETAILED EVALUATION

📖 METIN:
--------------------------------------------------------------------------------
Erkek ve işçi yaban arıları kış mevsiminden önce ölür, kraliçe yaban arıları ise kış uykusuna yatar. Bu durum onların aylarca soğuğa dayanmalarına ve baharda uyanarak yeni bir koloni kurmalarına olanak tanır. Kış boyunca hayatta kalan kraliçe arıların sayısı, ekosistemlerin sürdürülebilirliği ile doğrudan ilişkili olduğundan büyük önem taşır ve bu nedenle birçok çalışmanın odak noktasını oluşturur. Yakın zamanda yapılan bir deney esnasında da yanlışlıkla suya düşen kraliçe arıların önce öldüğü düşünülürken su boşaltıldığında uyandıkları ve herhangi bir zarar görmedikleri gözlemlenmiştir. Bu durum, ekosistem için önemli olan arıların olası bir su baskınına fiziksel olarak adapte olabilmelerinin mümkün olduğunu göstermiştir.

❓ SORU:
--------------------------------------------------------------------------------
Bu parçadan kraliçe yaban arılarıyla ilgili aşağ